# Démonstration du Evidence-Based Stock Screener

Ce notebook démontre l'utilisation du screener d'actions basé sur les facteurs Quality et Momentum, avec une attention particulière portée aux fonctionnalités de visualisation.

## 1. Configuration et importations

In [ ]:
# Imports standards
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from datetime import datetime, timedelta

# Ajout du chemin racine du projet pour pouvoir importer les modules
sys.path.append('..')

# Imports des modules du screener
from src.screener.screener import StockScreener
from src.visualization.chart_generator import ChartGenerator

# Configuration matplotlib et seaborn
plt.style.use('seaborn-whitegrid')
sns.set_palette('tab10')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100

# Pour afficher les graphiques Plotly dans le notebook
import plotly.io as pio
pio.renderers.default = 'notebook'

## 2. Initialisation du screener et chargement des données

In [ ]:
# Initialiser le screener
screener = StockScreener()

# Charger les données pour un indice
# Note: Cette étape peut prendre du temps si refresh_data=True
# Pour accélérer les tests, utilisez refresh_data=False si les données existent déjà
index_name = "SP500"  # Options possibles: "SP500", "NASDAQ", "EUROSTOXX", etc.
refresh_data = False  # Mettre à True pour télécharger des données fraîches

print(f"Chargement des données pour l'indice {index_name}...")
success = screener.load_index_data(index_name, refresh_data=refresh_data)

if success:
    print(f"Données chargées avec succès pour {index_name}")
else:
    print(f"Erreur lors du chargement des données pour {index_name}")

## 3. Calcul des scores Quality et Momentum

In [ ]:
# Calculer les scores avec une pondération personnalisée
# Vous pouvez ajuster les poids pour donner plus d'importance à un facteur
momentum_weight = 0.5  # Poids du facteur Momentum (0-1)
quality_weight = 0.5   # Poids du facteur Quality (0-1)

print("Calcul des scores Quality et Momentum...")
scores = screener.calculate_scores(momentum_weight=momentum_weight, quality_weight=quality_weight)

# Afficher les top stocks
top_n = 20  # Nombre de stocks à afficher
top_stocks = screener.get_top_stocks(n=top_n)

print(f"\nTop {top_n} stocks basés sur le score combiné:")
display(top_stocks)

## 4. Initialisation du générateur de graphiques

In [ ]:
# Créer le dossier de sortie pour les visualisations
output_dir = '../reports/visualizations'
os.makedirs(output_dir, exist_ok=True)

# Initialiser le générateur de graphiques
chart_gen = ChartGenerator(output_dir=output_dir)

## 5. Visualisation des scores par action

In [ ]:
# Visualiser les scores des meilleures actions
scores_fig = chart_gen.plot_factor_scores(
    scores_df=screener.combined_scores,
    top_n=15,  # Nombre d'actions à afficher
    title=f"Top stocks de l'indice {index_name} par facteurs Quality et Momentum",
    interactive=True  # True pour Plotly, False pour Matplotlib
)

# Afficher le graphique
scores_fig

## 6. Visualisation de la distribution par secteur

In [ ]:
# Obtenir les scores combinés avec les informations de secteur
combined_with_sector = top_stocks.copy()

# Visualiser la distribution par secteur
sector_fig = chart_gen.plot_sector_distribution(
    scores_df=combined_with_sector,
    top_n=30,  # Nombre d'actions à analyser
    title=f"Distribution sectorielle des meilleures actions - {index_name}",
    interactive=True
)

# Afficher le graphique
sector_fig

## 7. Carte de chaleur des facteurs

In [ ]:
# Visualiser les scores sous forme de heatmap
heatmap_fig = chart_gen.plot_factor_heatmap(
    scores_df=screener.combined_scores,
    top_n=50,  # Nombre d'actions à afficher
    title=f"Heatmap des facteurs Quality vs Momentum - {index_name}",
    interactive=True
)

# Afficher le graphique
heatmap_fig

## 8. Visualisation des performances historiques

In [ ]:
# Sélectionner quelques actions du top pour comparer leur performance
top_5_tickers = top_stocks.head(5).index.tolist()

# Ajouter l'indice comme référence (si disponible)
benchmark = 'SPY'  # Ou un autre ETF représentant l'indice

# Visualiser les performances
perf_fig = chart_gen.plot_performance_comparison(
    price_data=screener.price_data,
    tickers=top_5_tickers,
    lookback_days=365,  # 1 an de données
    benchmark_ticker=benchmark,
    title=f"Performance sur 1 an - Top 5 actions vs {benchmark}",
    interactive=True
)

# Afficher le graphique
perf_fig

## 9. Analyse des corrélations avec l'indice

In [ ]:
# Visualiser les corrélations roulantes avec l'indice
corr_fig = chart_gen.plot_rolling_correlations(
    price_data=screener.price_data,
    tickers=top_5_tickers,
    benchmark_ticker=benchmark,
    window=90,  # Fenêtre de 90 jours
    title=f"Corrélation roulante sur 90 jours avec {benchmark}",
    interactive=True
)

# Afficher le graphique
corr_fig

## 10. Comparaison de métriques fondamentales

In [ ]:
# Sélectionner une métrique fondamentale pour la comparaison
# Exemples de métriques possibles: 'roe', 'pe_ratio', 'debt_to_equity', etc.
# La disponibilité dépend des données collectées
metric = 'roe'  # Return on Equity

# Visualiser la comparaison de métriques
metric_fig = chart_gen.plot_metric_comparison(
    fundamental_data=screener.fundamental_data,
    tickers=top_5_tickers,
    metric_name=metric,
    title=f"Comparaison du {metric.upper()} pour les top 5 actions",
    interactive=True
)

# Afficher le graphique
metric_fig

## 11. Distribution des scores

In [ ]:
# Visualiser la distribution des scores
dist_fig = chart_gen.plot_score_distributions(
    scores_df=screener.combined_scores,
    title=f"Distribution des scores pour l'indice {index_name}",
    interactive=True
)

# Afficher le graphique
dist_fig

## 12. Filtrage avancé des actions

In [ ]:
# Filtrer les actions selon des critères spécifiques
filtered_stocks = screener.filter_stocks(
    min_momentum=60,  # Score Momentum minimum
    min_quality=70,   # Score Quality minimum
    sectors=['Technology', 'Healthcare']  # Secteurs à inclure
)

print(f"Actions filtrées (Momentum > 60, Quality > 70, secteurs Tech et Santé):")
display(filtered_stocks.head(10))

# Visualiser les actions filtrées
filter_fig = chart_gen.plot_factor_scores(
    scores_df=filtered_stocks,
    top_n=10,
    title="Top actions filtrées par critères spécifiques",
    interactive=True
)

# Afficher le graphique
filter_fig

## 13. Export des résultats

In [ ]:
# Exporter les résultats dans différents formats
# Excel (avec détails)
excel_path = screener.export_results(format='xlsx', include_details=True)
print(f"Résultats exportés en Excel: {excel_path}")

# CSV (format simple)
csv_path = screener.export_results(format='csv', include_details=False)
print(f"Résultats exportés en CSV: {csv_path}")

# HTML (pour intégration web)
html_path = screener.export_results(format='html', include_details=True)
print(f"Résultats exportés en HTML: {html_path}")

## Conclusion

Ce notebook a démontré comment utiliser le Evidence-Based Stock Screener pour:

1. Charger des données financières pour un indice spécifique
2. Calculer des scores basés sur les facteurs Quality et Momentum
3. Visualiser les résultats de différentes manières (scores, secteurs, performances, etc.)
4. Filtrer les actions selon des critères personnalisés
5. Exporter les résultats dans différents formats

Les visualisations interactives permettent d'explorer et d'analyser facilement les données pour prendre des décisions d'investissement mieux informées, basées sur des facteurs qui ont prouvé leur efficacité à long terme.